# Import packages and data

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
    AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import LabelEncoder
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder

from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
import shap

PREPROCESSED_FILE = '../data/preprocessed_train_val_Mar12_1130pm.csv'

df = pd.read_csv(PREPROCESSED_FILE, index_col='Reservation-id')

# RandomForest
# Extra Trees
# SVC
# LightGBM

X_train = df.drop(columns='Reservation_Status')
Y_train = df['Reservation_Status']

kfold = StratifiedKFold(n_splits=10)

random_state = 2


In [2]:
cv_results

NameError: name 'cv_results' is not defined

In [20]:
[x.mean() for x in cv_results]

[0.23041471238049746,
 0.1504272324866282,
 0.16711541580364891,
 0.15291330752069315]

In [13]:
[x.mean() for x in cv_results]

[0.2585997256658158,
 0.25766505487754066,
 0.2868909077515506,
 0.24170269609292255]

In [17]:
[x.min() for x in cv_results]

[0.22605016497596156,
 0.21092278719397362,
 0.28688010043942247,
 0.19200944414253787]

In [14]:
[x.std() for x in cv_results]

[0.02923751581925276,
 0.031138680758617115,
 2.1614624256138803e-05,
 0.035678579965796424]

In [3]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)

In [6]:
X_train

array([[-0.2613606 , -0.28358421,  0.3550371 , ...,  0.57383819,
         0.70870286,  0.63336308],
       [ 0.32708497,  0.56579247,  1.73966347, ..., -1.74265154,
         0.70870286,  0.63336308],
       [-0.13059492,  0.56579247,  1.73966347, ..., -1.74265154,
         0.70870286,  0.63336308],
       ...,
       [-1.6997831 , -0.28358421,  1.73966347, ...,  0.57383819,
         0.70870286, -1.57887322],
       [-0.13059492,  2.26454583, -1.02958927, ...,  0.57383819,
        -1.41102859,  0.63336308],
       [ 0.58861633,  1.41516915, -1.02958927, ...,  0.57383819,
         0.70870286, -1.57887322]])

In [7]:
cb = CatBoostClassifier(max_depth=10, auto_class_weights='Balanced', n_estimators=10)
# svc = SVC(class_weight='balanced')
knn = KNeighborsClassifier()
et = ExtraTreesClassifier(max_depth=10, n_estimators=10, class_weight='balanced')
ab = AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state, max_depth=10, 
                                               class_weight='balanced'),
                        random_state=random_state,learning_rate=0.1)
mlp = MLPClassifier()

In [32]:
model = VotingClassifier([('catboost', cb),
                          ('SVC', svc), 
                          ('KNN', knn), 
                          ('Extra Trees', et), 
                          ('AdaBoost', ab),
                          ('MLP', mlp)], n_jobs=-1, verbose=True)

In [6]:
cv_score = cross_val_score(ab,
                          X_train, y = Y_train, 
                          scoring = make_scorer(f1_score, average='macro'), 
                          cv = kfold, n_jobs=-1)

In [7]:
cv_score

array([0.29571927, 0.28525755, 0.28471109, 0.27897608, 0.29387609,
       0.26527577, 0.25853374, 0.2721672 , 0.24946889, 0.2948778 ])

In [ ]:
model.fit(X_train, Y_train)